## Conversion to ONNX from keras model using tf2onnx python api 

In [1]:
# get image
# !wget -q https://github.com/onnx/tensorflow-onnx/raw/master/tests/ade20k.jpg

In [2]:
# !pip install tensorflow tf2onnx

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import onnxruntime

img_path = 'ade20k.jpg'

img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

### Run the keras model

In [7]:
# 創建 model 
model = ResNet50(weights='imagenet')
model._name = "tensorflow_ResNet50"
print(model.name)
# 確定 model 功能正常
preds = model.predict(x)
print('Keras Predicted:', decode_predictions(preds, top=3)[0])
# 確定 model 可以存
# model.save(os.path.join("./temp/"+model.name))

tensorflow_ResNet50
Keras Predicted: [('n04285008', 'sports_car', 0.34311852), ('n02974003', 'car_wheel', 0.288192), ('n03100240', 'convertible', 0.101694115)]


### Convert to ONNX using the Python API

In [ ]:
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = "./" + model.name + ".onnx"
print(output_path)

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 320 variables.
INFO:tensorflow:Converted 320 variables to const ops.



Removing optional input keras_learning_phase that appears to be a keras learning phase parameter. Use --ignore_default to force this into an input.
Removing optional input keras_learning_phase that appears to be a keras learning phase parameter. Use --ignore_default to force this into an input.


### Run the ONNX model

In [ ]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {})

print('ONNX Predicted:', decode_predictions(onnx_pred[0], top=3)[0])

# make sure ONNX and keras have the same results
np.testing.assert_allclose(preds, onnx_pred[0], rtol=1e-5)

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid Feed Input Name:input

### Convert to ONNX using the command line

In [ ]:
# !python -m tf2onnx.convert --opset 13 \
#     --saved-model {os.path.join("/tmp", model.name)} \
#     --output  {os.path.join("/tmp", model.name + ".onnx")}

/opt/anaconda3/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-02-22 16:19:56,658 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2021-02-22 16:20:01,809 - INFO - Signatures found in model: [serving_default].
2021-02-22 16:20:01,809 - WARNING - '--signature_def' not specified, using first signature: serving_default
2021-02-22 16:20:01,810 - INFO - Output names: ['predictions']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-02-22 16:20:03,659 - WARNING - From /home/ipython/.local/lib/python3.7/site-packages/tf2onnx/tf_loader.py:558: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-02-22 16:20:0